# Idea of the Wrapping Approach
- running (or loading a previously ran) a disjointification model to narrow down number of features from thousands to a few hundreds
- further narrow the number of features to ~100 by train-test-validation splitting and running a basic selection algorithm

## Imports / Defs

In [44]:
import disjointification
import utils
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from pathlib import Path
save_point = Path(r"disjointification_model_point_bi_serial_fulldataset/09_09_2023__01_24_48/09_09_2023__01_24_48.pkl")
if 'disj' not in locals():
    disj = disjointification.from_file(save_point)
disj.describe()

Disjointification Test Description
features data: (3069, 9259)
labels data: (3069, 2)
regression label: Lympho
classification label: ER
correlation method regression: pearson
correlation method regression: <function point_bi_serial_r_correlation at 0x0000021662E20430>
min num of features to keep in disjointification: 200
correlation threshold: 0.4
last save point: \model\09_09_2023__01_24_48\09_09_2023__01_24_48.pkl
number of features kept in disjointification: lin 200, log 200


In [51]:
disj.features_df.shape

(3069, 9259)

In [52]:
disj.labels_df.shape

(3069, 2)

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LinearRegression, LogisticRegression

regression_label = disj.lin_regressor_label
classification_label = disj.log_regressor_label
validation_size_split = 0.3
test_size_split = 0.2

train_size_split = 1-validation_size_split-test_size_split
actual_test_size_split = test_size_split/(test_size_split+train_size_split)

In [54]:
y = disj.labels_df[classification_label].copy().dropna()
x = disj.features_df.copy().dropna()
x_train_test, x_validation, y_train_test, y_validation = train_test_split(x, y, test_size=validation_size_split, random_state=97)
x_train, x_test, y_train, y_test = train_test_split(x_train_test, y_train_test, test_size=validation_size_split, random_state=34)

In [55]:
skb = SelectKBest(k=10)
selected_features_classification = skb.fit_transform(x_train, y_train)

In [56]:
lr = LogisticRegression(max_iter=10000)
lr.fit(x_test, y_test)

LogisticRegression(max_iter=10000)

## Repeat For Regression